In [ ]:
# 이 글은 YOUTUBE 파이토치 입문 강의를 수강하고 작성한 문서입니다.
# https://www.youtube.com/watch?v=8PnxJ3s3Cwo&t=888s&ab_channel=%EB%94%A5%EB%9F%AC%EB%8B%9D%ED%98%B8%ED%98%95DLbro

(요약)
### 1. 파이토치 제공 데이터 사용
### 2. 같은 클래스 별 폴더 이미지 데이터 이용
### 3. 개인 데이터 사용 (2 types)

In [2]:
import torch 
import torchvision
import torchvision.transforms as tr 
# torchvision.transforms: 데이터를 불러오면서 전처리를 바로 할 수 있도록 해준다.
from torch.utils.data import DataLoader, Dataset 
# DataLoader: 배치사이즈 형태로 만들어서 실제로 학습을 할 떄 이용할 수 있는 형태로 만들어주는 라이브러리
# Dataset: 튜닝 할 때 사용 
import numpy as np

#import pandas as pd
#print("pandas version: ", pd.__version__)
#pd.set_option('display.max_row', 500000)
#pd.set_option('display.max_columns', 100000)

# 1. 파이토치 제공 데이터 사용
- 많은 데이터가 있으며 그 중 원하는 것으로 파일명을 적으면 된다. 
- 실제 논문에서 많이 사용되는 데이터이기 때문에 신뢰성이 높다.

In [3]:
# 파이토치 사이트에 가면 여러가지 정의가 있다.
# 그 중에 필요한 전처리 작업을 일렬로 compose안에 나열해주자.
# compose: 전처리를 할 때 순서대로 작업을 처리하게 된다.
# 지금의 경우는 8 by 8 로 resize가 되고, tensor데이터로 바꿔준다는 뜻
transf = tr.Compose([tr.Resize(8),tr.ToTensor()])

# Transforms on PIL Image 
# ***(주의)처음의 이미지는 PIL 이미지라고 해서 특정 타입을 이야기한다. 여기서 이 부분 주의하자. 
# numpy, list transform에 넣어주면 오류가 난다. 즉, PIL 이미지여야 한다.

# Pad, Grayscale, RandomCrop, Normalize ..
# Transforms on torch.*Tensor - tensor image
# torchvision.transforms.ToPILImage(mode=None)...

In [4]:
# root: 다운로드를 받을 수 있는 경로
# download: 당연히 받아야 한다.
# transform: 다운로드를 받고 들어오는 데이터에 대해 전처리를 해주겠다.
trainset = torchvision.datasets.CIFAR10(root='./data',train=True, download=True, transform=transf)
testset = torchvision.datasets.CIFAR10(root='./data',train=False, download=True, transform=transf)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
# 트레이닝 셋의 사이즈를 보자.
# 튜플형태로 들어가 있다.
# 첫번째로 이미지 그 옆으로는 레이블이 들어가 있다.
print(trainset[0][0].size())
print(trainset[0][0])
# 의미: 채널 3개의 8 x 8짜리 이미지다.
# CIFAR10이 RGB채널이라 3채널이 된다. 

# 일반적으로 이미지는 이미지 크기와 그 뒤에 채널 갯수가 붙게 된다. 
# 그래서 PYTHON이나 OpenCV(Open Source Computer Vision) 이런 것들을 이용할 떄 
# ([8, 8, 3])으로 표현되어 있다. 파이토치는 다름

torch.Size([3, 8, 8])
tensor([[[0.2275, 0.4510, 0.4667, 0.4745, 0.4745, 0.5098, 0.5255, 0.4824],
         [0.4196, 0.4902, 0.4627, 0.3882, 0.4275, 0.4275, 0.4902, 0.3765],
         [0.5529, 0.5020, 0.4549, 0.4549, 0.5843, 0.4941, 0.4980, 0.5059],
         [0.5608, 0.4235, 0.5765, 0.7804, 0.8510, 0.7765, 0.5922, 0.5176],
         [0.5294, 0.5412, 0.7529, 0.7686, 0.7333, 0.7804, 0.6392, 0.5059],
         [0.5333, 0.6118, 0.7216, 0.6667, 0.5765, 0.5804, 0.5059, 0.5216],
         [0.6196, 0.5765, 0.6196, 0.6157, 0.5569, 0.5294, 0.5255, 0.5922],
         [0.7294, 0.6706, 0.5804, 0.5608, 0.5647, 0.5294, 0.4667, 0.4863]],

        [[0.1529, 0.3059, 0.3255, 0.3294, 0.3333, 0.3686, 0.3765, 0.3490],
         [0.2784, 0.3176, 0.3059, 0.2431, 0.2627, 0.2784, 0.3255, 0.2471],
         [0.4039, 0.3529, 0.3020, 0.2863, 0.3608, 0.3294, 0.3333, 0.3490],
         [0.4118, 0.2902, 0.4118, 0.6314, 0.7333, 0.6627, 0.4588, 0.3725],
         [0.3647, 0.4118, 0.6314, 0.6431, 0.6392, 0.7059, 0.5294, 0.3922],
 

In [8]:
# 데이터 셋을 받아서 이용할 수 있는 배치 형태로 만들어 보자.
# num_workers: 데이터를 로드할 때 프로세스를 몇개 쓰느냐. ex)0, 2, 4...
# 데이터 로더를 통해서 배치형태로 전부 분리 상태. 
# 후 테스트 로더를 통해서 인공신경망에 바로 사용 가능.
trainloader = DataLoader(trainset, batch_size=50, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=50, shuffle=True, num_workers=2)

In [9]:
# CIFAR10 데이터의 트레이닝 이미지 개수가 50,000개이기 때문에 batch_size=50을 설정하면
# batch_size=50개 짜리가 1000개가 있다는 뜻.
# 확인해보면 50,000개를 잘 나눈다면 데이터 로더의 길이가 1000이다.
len(trainloader)

1000

In [10]:
# iter(), next(): trainloader의 실제 값을 보기 위한 함수
# iter : 하나씩 불러오겠다.
# .next(): 한 묶음에 대해서 불러 온다고 생각하자.
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [11]:
print(dataiter)
print(images[0][0])
print(labels)

tensor([[0.6118, 0.6314, 0.6588, 0.6706, 0.6667, 0.6549, 0.6353, 0.6118],
        [0.6078, 0.6078, 0.6980, 0.7137, 0.7137, 0.6980, 0.6824, 0.6627],
        [0.5882, 0.5333, 0.6863, 0.7294, 0.7451, 0.7373, 0.7216, 0.7020],
        [0.5843, 0.4314, 0.4588, 0.6039, 0.7412, 0.7608, 0.7451, 0.7294],
        [0.5686, 0.3255, 0.2980, 0.3961, 0.6157, 0.7490, 0.7451, 0.7333],
        [0.6392, 0.5255, 0.4549, 0.4588, 0.5176, 0.6667, 0.7098, 0.7176],
        [0.6706, 0.6784, 0.6863, 0.6745, 0.6431, 0.6078, 0.5804, 0.6471],
        [0.6627, 0.6784, 0.6941, 0.7020, 0.7020, 0.6902, 0.6667, 0.6667]])
tensor([0, 4, 9, 0, 7, 8, 7, 4, 3, 3, 5, 9, 9, 4, 3, 5, 2, 8, 5, 4, 0, 8, 3, 8,
        7, 8, 5, 9, 9, 6, 2, 9, 3, 6, 7, 6, 6, 0, 0, 6, 2, 5, 7, 6, 5, 2, 3, 9,
        9, 8])


In [12]:
# 항상 파이토치는 신경망에 들어갈 떄 이 순서이다. 잘 기억하자.
# batch_size가 50, 채널 3개, 이미지 너비가 8, 높이가 8
images.size()

torch.Size([50, 3, 8, 8])

# 2. 같은 클래스별 폴더 이미지 데이터 이용

In [ ]:
# 코드가 있는 폴더에 class폴더를 만들고 그 안에 클래스마다 폴더 생성 후 이미지를 넣어야한다.
# 원하는 이미지를 사용할 수 있도록 하는 방법이다.

# 정리를 잘 해서 이미지 데이터를 폴더 안에 class 별로 다 나눠놓은 경우...
# ./class/tiger(이미지A)   ./class/lion(이미지B)
# 예를 들어 위와 같이 class 폴더 안에 tiger 폴더와 lion 폴더가 있다.
# 폴더가 깔끔하게 나눠진 상태에서 torchvision.datasets.ImageFolder를 사용하여 
# root='./class' 폴더 안의 이미지를 알아서 search해 주고 각각의 다른 폴더에 대한
# 레이블링을 다르게 자동으로 입력해준다. (+transform을 통해 전처리도 이용가능)
# 이미지 명과 관계없이 폴더와 파일 정리만 잘 해주면 알파벳 순으로 들어온다.
# 즉 한 줄로 데이터 전체를 불러오고 레이블이 자동으로 매겨지면서 전처리까지 가능하다.
transf = tr.Compose([tr.Resize(16),tr.ToTensor()])
trainset = torchvision.datasets.ImageFolder(root='./class', transform=transf)

trainset[0][0].size()
# ex> torch.Size([3,16,16])
# 채널 수 3개이며 Resize(16)이기때문에 16 by 16

# 그리고 실제 학슴을 할 수 있게 배치 사이즈 형태로 데이터 로더를 통해 만들자.
trainloader = DataLoader(trainset, batch_size=10, shuffle=False, num_workers=2)
print(len(trainloader))
# ex) 3일경우 batch_size 10 짜리가 3개 즉, 예시로는 30개의 이미지

#### 그럼에도 불구하고 개인적으로 torchvision.datasets.ImageFolder를 쓰지 않는 이유

- 3번처럼 할 수 있어야하는 이유: 첫번째로 클래스별로 폴더를 나눌 수 있다면 2번처럼<br>
transf = tr.Compose([tr.Resize(16),tr.ToTensor()])<br>
trainset = torchvision.datasets.ImageFolder(root='./class', transform=transf)<br>
trainloader = DataLoader(trainset, batch_size=10, shuffle=False, num_workers=2)<br>
불러오고 전처리하고 배치 나누는 것까지 3줄이면 끝낼 수 있다. 하지만 이렇게 만들 수 없는 경우가 있다. <br>
<br>
- 폴더 정리를 못하는 경우 
1. 다른 작업과 공용으로 사용하는 경우  
2. 폴더가 아닌 SQL같은 곳에서 넘어오는 경우 함부로 디렉토리를 바꿀 수 없다.

- 파이토치에서 제공하는 Transform의 종류가 제한적이다.
예를 들어서 openCV같은 경우 종류가 훨씬 많고 직접 이미지 전처리에 대한 함수를 만들어야할 떄가 있다. 이런 디테일한 부분에 대해서 파이토치에서 제공한 전처리 작업은 사용하지 않는다. 그렇기 때문에 데이터 전처리는 모듈을 만들어 놓는다...
<br><br>
아래의 경우 한줄로 만들어 preprocessing 이라는 다른 파일에서 전처리가 다 되고 넘어온다.<br>
즉, tensor로 바꾸기 전에 전처리가 전부 끝나고 나서 바꾸게 된다.<br>
train_images, train_labels = preprocessing(train_images, train_labels)


# 3. 개인 데이터 사용 (2 types)

In [55]:
#import preprocessing

# 먼저, numpy형태의 데이터가 들어왔다고 가정하자.
# 그래서 32x32짜리 3채널짜리 이미지가 20개 있다고 가정한다.
train_images = np.random.randint(256, size=(20,32,32,3))
# 숫자 0, 1 그에 따른 레이블 20개가 있다고 가정하자.
train_labels = np.random.randint(2, size=(20,1))

# preprocessing 이라는 다른 파일에서 전처리가 다 되고 넘어온다.
# 즉, tensor로 바꾸기 전에 전처리가 전부 끝나고 나서 바꾸게 된다.
#train_images, train_labels = preprocessing(train_images, train_labels)

print(train_images.shape,'\n', train_labels.shape)

(20, 32, 32, 3) 
 (20, 1)


In [23]:

# 1. 상속을 받을 클래스를 생성.
class TensorData(Dataset):
    
    # 정형화된 함수 3개
    # 외부에서 데이터가 들어오도록 x_data, y_data 설정(없어도 된다.)
    # def __init__(self):
    def __init__(self, x_data, y_data):
        # 데이터를 텐서로 전환 (float, long과 같이 숫자 속성부여)
        self.x_data = torch.FloatTensor(x_data)
        # permute함수를 활용하여 순서 변환
        # 이미지 개수(배치), 채널 수, 이미지 너비, 높이로 맞추자.
        self.x_data = self.x_data.permute(0,3,1,2) 
        self.y_data = torch.LongTensor(y_data)
        # 데이터의 개수산출하기
        self.len = self.y_data.shape[0]
    
    # x, y를 튜플 형태로 바깥으로 내보내기 위함
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

In [24]:
# 데이터 생성
train_data = TensorData(train_images, train_labels)
# 데이터 로더
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)

In [25]:
# 순서가 바뀌었음을 보자.
train_data[0][0].size()

torch.Size([3, 32, 32])

In [28]:
# 마찬가지로 iter를 통해 사이즈를 보자
dataiter = iter(train_loader)
images, labels = dataiter.next()

In [57]:
# batch_size=10, 채널 3개, 이미지 사이즈 32x32
images.size()

torch.Size([10, 3, 32, 32])

### 꿀팁 방출!
# 4. Pytorch로 무조건 전처리하기
클래스로 나눠놓지 못한 경우 개인 데이터에 transform 이용하기.<br><br>
(이 형태를 하나의 양식이라 생각하고 작업한다.)
- import torch.utils.data import Dataset
- class MyDataset(Dataset):
-- def __init__(self):
-- def __getitem__(self, index):
-- def __len__(self):

In [60]:
class MyDataset(Dataset):
    
    # transform=None <- 어떠한 명령어를 적용하여 작동하도록
    def __init__(self, x_data, y_data, transform=None):
        
        # 위와 같이 self.x_data = torch.FloatTensor(x_data)로 
        # 텐서변환 작업을 하지 않았기 떄문에 numpy형태로 나온다.
        self.x_data = x_data
        self.y_data = y_data
        self.transform = transform
        self.len = len(y_data)
    
    def __getitem__(self, index):
        sample = self.x_data[index], self.y_data[index]
        
        # 튜플 형태로 내보내기 전에 전처리 작업을 하겠다.
        # transform=None이면 작업을 안하고 넘어가는 것이고...
        if self.transform:
            sample = self.transform(sample)
        
        return sample
    
    def __len__(self):
        return self.len
    
# =========================================================
# 위의 transform을 메뉴얼로 작성해보자
# ==========================================================  
# 텐서를 바꿔주는 클래스를 만들어서 위의 self.x_data = torch.FloatTensor(x_data)에서
# 텐서를 바꾸지 않고 실제 transform이 연산되는 과정과 똑같이 만들어 보자.
# __call__함수를 만들어 sample을 받게끔 하자.

class ToTensor:
    def __call__(self, sample):
        inputs, labels = sample
        # input을 floattensor로 만들고
        inputs = torch.FloatTensor(inputs)
        # 형태(채널, 이미지 사이즈)를 바꾸기 위해 permute를 쓴다.
        inputs = inputs.permute(2,0,1)
        # labels은 그냥 텐서로 바꿔주고...
        return inputs, torch.LongTensor(labels)
    
# ==========================================================
# 이제 어떤 연산을 한다... nomalization도 그렇고 ..
# 만약 어떤 데이터가 들어오면 거기에 어떤 상수를 곱하고 상수를 더하는 형태를 만들고 싶을 떄
# slop와 bias를 정해져서 받을 수 있도록 init에 세팅하고 
# 샘플 불러올 수 있게 call함수를 적어야 한다.
# ==========================================================
class LinearTensor:
    
    def __init__(self, slope=1, bias=0):
        self.slope = slope
        self.bias = bias
    
    def __call__(self, sample):
        inputs, labels = sample
        inputs = self.slope*inputs + self.bias
        
        return inputs, labels

In [59]:
# 사용방법 똑같다. 
# compose를 통해 지정한다.
# 헷갈리지말자. 
# 앞전의 ToTensor는 tr.ToTensor로써 torchvision.transform의 함수를 사용하는 것 
trans = tr.Compose([ToTensor(),LinearTensor(2,5)])
ds1 = MyDataset(train_images, train_labels, transform=trans)
train_loader1 = DataLoader(ds1, batch_size=10, shuffle=True)

In [ ]:
first_data = ds1[0]
features, 